In [1]:
import pandas as pd

# ---- Load ----
highs = pd.read_csv("highs.csv", parse_dates=["Date"])
lows = pd.read_csv("lows.csv", parse_dates=["Date"])
market_caps = pd.read_csv("monthly_market_caps.csv", parse_dates=["Date"])
volumes = pd.read_csv("monthly_average_volumes.csv", parse_dates=["Date"])

# Extract company columns (all columns except Date)
company_cols = highs.columns[1:]

# Add Year column to each
for df in [highs, lows, market_caps, volumes]:
    df["Year"] = df["Date"].dt.year


In [2]:
yearly_lows = lows.groupby("Year")[company_cols].min()
yearly_highs = highs.groupby("Year")[company_cols].max()
yearly_caps = market_caps.groupby("Year")[company_cols].max()
yearly_vols = volumes.groupby("Year")[company_cols].mean()


In [3]:
# Convert datasets to long (Year, Company, value)
low_long = yearly_lows.reset_index().melt("Year", var_name="Company", value_name="LowestLow")
high_long = yearly_highs.reset_index().melt("Year", var_name="Company", value_name="HighestHigh")
cap_long = yearly_caps.reset_index().melt("Year", var_name="Company", value_name="MaxMarketCap")
vol_long = yearly_vols.reset_index().melt("Year", var_name="Company", value_name="AvgVolume")

# Merge all together
df = low_long.merge(high_long, on=["Year", "Company"])
df = df.merge(cap_long, on=["Year", "Company"])
df = df.merge(vol_long, on=["Year", "Company"])


df["Year"] = df["Year"].astype(int)

In [4]:
import altair as alt

# Slider binding
slider = alt.binding_range(
    min=df["Year"].min(),
    max=df["Year"].max(),
    step=1,
    name="Year:"
)

# Create param correctly
year_select = alt.param(
    name="year_select",
    bind=slider,
    value=df["Year"].min()
)

chart = (
    alt.Chart(df)
    .mark_circle()
    .encode(
        x=alt.X("LowestLow:Q", title="Lowest Low (Yearly)"),
        y=alt.Y("HighestHigh:Q", title="Highest High (Yearly)"),
        size=alt.Size("MaxMarketCap:Q",
                      title="Max Market Cap",
                      scale=alt.Scale(range=[10, 1000])),
        color=alt.Color("AvgVolume:Q", title="Avg Volume"),
        tooltip=["Company", "LowestLow", "HighestHigh", "MaxMarketCap", "AvgVolume", "Year"]
    )
    # --- key fix: the new syntax uses alt.datum.year_select ---
    .transform_filter(alt.datum.Year == year_select)
).add_params(
    year_select
).properties(
    width=600,
    height=400,
    title="Tech Companies: High vs Low (Market Cap + Volume)"
)

chart

alt.Chart(...)